# Лабораторная работа №9
## Создание текстового файла большого объема с помощью генераторной функции

Булгаков А.Е. 31.05.2025

**Используемые модули:** `random`, `time`

### Задание 9.1
Создание и запись строк в текстовый файл.

In [22]:
file_name = 'test.txt'
f = open(file_name, 'w')
lines_list = [f'{x}\n' for x in 'test']
f.writelines(lines_list)
f.close()

with open(file_name,'r') as f:
    for line in f:
        print(line, end='')

t
e
s
t


In [23]:
lines_gen_expr = (f'{x}\n' for x in range(10**6))

def lines_gen_fun(number_lines=10**6):
    yield from (f'{x}\n' for x in range(number_lines))

with open(file_name,'w') as f:
    f.writelines(lines_gen_expr)
    f.writelines(lines_gen_fun())

In [24]:
def lines_gen_fun(number_lines=10**6):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line.encode('utf8'))
    print(f'{file_size / 1024**2:.2f} Mb')

with open(file_name, 'w') as f:
    f.writelines(lines_gen_fun())

6.57 Mb


### Задание 9.2
Генерация случайных строк с использованием различных типов символов

In [25]:
from random import randint

latin_code = (ord('a'), ord('z'))
number_words = (10, 20)
number_symbols = (5, 10)

len_word = randint(*number_symbols)
word = ''.join([chr(randint(*latin_code)) for _ in range(len_word)])
print(word)

uugxns


In [ ]:
def lines_gen_fun_v2(file_size, type_symbols='latin', number_words=(10,20), number_symbols=(5,10), status=True):
    """
    Генератор строк, суммарный объём которых составляет указанный размер в мегабайтах

    file_size- желаемый объём данных в мегабайтах
    type_symbols -тип символов(latin,digits,cyrillic)
    number_words -диапазон количества слов в строке (min, max)
    number_symbols- диапазон длины слов в символах (min, max)
    status -отображать ли прогресс выполнения в процентах
    yield- сгенерированная строка текста
    """

    if type_symbols == 'latin': #определяет диапавзон кодов символов в зависимости от типа
        code_range = (ord('a'), ord('z'))  #латиница a-z
    elif type_symbols == 'digits':
        code_range = (ord('0'), ord('9'))  # 0-9
    elif type_symbols == 'cyrillic':
        code_range = (ord('а'), ord('я'))  #кирилица а-я

    total_bytes=0   #количество записанных байтов
    target_bytes=file_size*1024*1024       #желаемфй размер файла в байтах  
    percent_prev = -1  #процент вывенный раньшен

    while total_bytes < target_bytes:   #генерация до нужного размера
        words = []  #список слов в строке
        for _ in range(randint(*number_words)):  #рандомное количество слов
            length = randint(*number_symbols)       #рандомная длина слова
            word = ''.join(chr(randint(*code_range)) for _ in range(length)) #слово из рандомных символов выбранного диапазона
            words.append(word)  #добавить слово в список

        line = ' '.join(words) + '\n'       # объединить слова в строку и добавляем перевод строки
        total_bytes += len(line.encode('utf8'))      #учитываем размер строки в байтах

        if status: #выводим процент выполнения
            percent = int(total_bytes / target_bytes * 100)
            if percent != percent_prev:
                print(f'\r{percent}%', end='', flush=True)
                percent_prev = percent

        yield line  #вернутьь строку

    print(f'\n{total_bytes / 1024**2:.2f} Mb')#размер файла

### Задание 9.3
Финальная функция генерации и записи файла.

In [ ]:
from typing import Tuple
from time import process_time

def file_gen(file_name: str,
             file_size: int,
             type_symbols: str = 'latin',
             number_words: Tuple[int, int] = (10, 20),
             number_symbols: Tuple[int, int] = (5, 10)):
    """
    Генерация текстового файла заданного размера с помощью генераторной функции строк

    file_name -имя выходного текстового файла
    file_size- желаемый размер файла в мегабайтах
    type_symbols -тип символов (latin,digits,cyrillic)
    number_words- диапазон количества слов в строке(min, max)
    number_symbols- диапазон длины каждого слова (min, max)
    """
    start = process_time()  #начальное время 
    with open(file_name, 'w', encoding='utf8') as f:  #открыввает файл для записи в кодировке UTF-8
        f.writelines(lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols)) #записывает сгенерированные строки в файл
    end = process_time()  #время окончания
    print(f'Время выполнения: {end-start:.2f} секунд')  #вывод времени

In [28]:
file_gen('latin_test.txt', 1, 'latin')
file_gen('digits_test.txt', 1, 'digits')
file_gen('cyrillic_test.txt', 1, 'cyrillic')

100%
1.00 Mb
Время выполнения: 0.38 секунд
100%
1.00 Mb
Время выполнения: 0.38 секунд
100%
1.00 Mb
Время выполнения: 0.30 секунд
